# Chapter 5: Quantum Annealing Linear Solvers

Introduction to quantum annealing using D-Wave systems and QUBO formulations.

---

**Prerequisites:**
- Python 3.8+
- Qiskit 2.1.2
- See `Chapter02_Software.ipynb` for installation instructions


In [1]:
# Setup and imports for D-Wave
import numpy as np
import matplotlib.pyplot as plt
from pyqubo import Binary, Array,Placeholder
from dimod.reference.samplers import ExactSolver
import neal
import networkx as nx

from Chapter03_TrussOptimization_functions import TrussFEM
from Chapter05_QALinearSolver_functions import TrussQUBOOptimizer, QUBOBoxSolverClass
print('Setup complete!')

Setup complete!


## QUBO Box Solver
The Box algorithm to solve a linear system of equations.

## Example to illustrate the Box algorithm

$$ \mathbf {A} \mathbf {x} = \mathbf {b} $$


$$ \mathbf {A} =  \begin{bmatrix}
            6 & -6 & 0 & 0 & 0 \\
             -6 & 12 & -6 & 0 & 0 \\
            0 & -6 & 12 & -6 & 0 \\
            0 & 0 & -6 & 12 & -6 \\
             0 & 0 & 0 & -6 & 12 \\
            \end{bmatrix} $$
$$ \mathbf {b}  = \mathbf {A} \begin{pmatrix} \pi/3 \\ -1/2 \\ 1/3 \\ \pi/10 \\ 0.1 \\0.5 \end{pmatrix} $$


In [2]:

A = np.array([[6,-6,0,0,0,0],[-6,12,-6,0,0,0],[0,-6,12,-6,0,0],[0,0,-6,12,-6,0],
		      [0,0,0,-6,12,-6],[0,0,0,0,-6,12]])
xExact = np.array([np.pi/3,-1/2,1/3,-np.pi/10,0.1,0.5 ])
b = A.dot(xExact)

QB = QUBOBoxSolverClass(
	beta=0.5,
	LBox0=1,
	tol=1e-8,
	samplingMethod="simulatedAnnealing",
	nSamples=100,
	boxMaxIteration=100
	)

xSolution, LHistory1, nIterations, _, _, _, _ = QB.QUBOBoxSolve(A, b)
print("xExact:", xExact)
print("xSolution:", xSolution)
print("*" * 50)

xExact: [ 1.04719755 -0.5         0.33333333 -0.31415927  0.1         0.5       ]
xSolution: [ 1.04736328 -0.49975586  0.33349609 -0.31396484  0.10009766  0.5       ]
**************************************************


## QUBO Truss Solver

In [3]:
nodes = np.array([
    [0.0, 0.0], [2.0, 0.0], [4.0, 0.0],
    [0.0, 1.5], [2.0, 1.5], [4.0, 1.5],
    [0.0, 3.0], [2.0, 3.0], [4.0, 3.0]
])

elements = [
    (0, 1), (1, 2), (3, 4), (4, 5), (6, 7), (7, 8),  # Horizontal
    (0, 3), (3, 6), (1, 4), (4, 7), (2, 5), (5, 8),  # Vertical
    (0, 4), (1, 3), (1, 5), (2, 4), (3, 7), (4, 6), (4, 8), (5, 7),  # Diagonals
    (0, 8), (2, 6), (0, 7), (1, 6), (1, 8), (2, 7)   # Long diagonals
]

# Create FEM model
fem_model = TrussFEM(nodes, elements, E=200e9, A=0.001, rho=7850)

# Problem setup
fixed_dofs = [0, 1, 4, 5]
loads = np.zeros(2 * len(nodes))
loads[2*7 + 1] = -10000  # 10 kN downward at node 7

# Create QUBO optimizer
optimizer = TrussQUBOOptimizer(fem_model, loads, fixed_dofs)

# Method 1: Exact solver (only for small problems, N < 20)
if len(elements) <= 15:
    print("\n" + "="*60)
    print("METHOD 1: EXACT QUBO SOLVER")
    print("="*60)
    best_design, best_metrics, all_results = optimizer.solve_exact()
    
    if best_design is not None:
        u, _ = fem_model.solve(best_design, loads, fixed_dofs)
        fem_model.plot_truss(design=best_design, loads=loads, 
                            fixed_dofs=fixed_dofs, displacements=u, 
                            scale_factor=50, show_labels=True,
                            title="Optimal Design (Exact QUBO Solver)",
                            save_path="truss_qubo_exact.png")

# Method 2: Simulated Annealing (for larger problems)
print("\n" + "="*60)
print("METHOD 2: SIMULATED ANNEALING")
print("="*60)
best_design, best_metrics, history = optimizer.solve_annealing(
    num_reads=200,
    num_iterations=3
)

if best_design is not None:
    u, _ = fem_model.solve(best_design, loads, fixed_dofs)
    fem_model.plot_truss(design=best_design, loads=loads, 
                        fixed_dofs=fixed_dofs, displacements=u, 
                        scale_factor=50, show_labels=True,
                        title="Optimal Design (Simulated Annealing)")
    
    


METHOD 2: SIMULATED ANNEALING
Solving with Simulated Annealing...
  3 iterations × 200 reads/iteration

Iteration 1/3
  Evaluating 200 unique solutions...


TypeError: TrussFEM.evaluate_design() got an unexpected keyword argument 'u_hat'. Did you mean 'd_hat'?